<img src="../logo_UTN.svg" align="right" width="150" /> 

#### Procesamiento Digital de Señales

# Trabajo Práctico Nº9
#### Federico Borello

# Filtrado Digital

Se sigue la [guía de laboratorio](https://nbviewer.org/github/marianux/jupytest/blob/master/Laboratorio%20de%20filtrado%20digital.ipynb) propuesta por la cátedra.

# Seteo las Señales

In [1]:
# Audio, ECG, PPG y Código original de la cátedra:
# https://nbviewer.org/github/marianux/jupytest/blob/master/Laboratorio%20de%20filtrado%20digital.ipynb

import scipy.signal as sig
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.io as sio
from pytc2.sistemas_lineales import plot_plantilla

fig_sz_x = 10
fig_sz_y = 7
fig_dpi = 100  # dpi

fig_font_size = 16

mpl.rcParams["figure.figsize"] = (fig_sz_x, fig_sz_y)
plt.rcParams.update({"font.size": fig_font_size})

sio.whosmat("../TS8/ECG_TP4.mat")
mat_struct = sio.loadmat("../TS8/ECG_TP4.mat")

ecg_one_lead = mat_struct["ecg_lead"]
ecg_one_lead = ecg_one_lead.flatten()
cant_muestras = len(ecg_one_lead)

fs = 1000  # Hz
nyq_frec = fs / 2

/home/fb/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [ ]:
plt.figure(figsize=(24, 8))
plt.plot(ecg_one_lead[:30000])
plt.title("ECG One Lead")
plt.show()

# Plantilla de Filtro con pytc2

In [ ]:
# Plantilla

# filter design
ripple = 0.1  # dB
atenuacion = 40  # dB

ws1 = 1.0  # Hz
wp1 = 3.0  # Hz
wp2 = 25.0  # Hz
ws2 = 35.0  # Hz

frecs = np.array([0.0, ws1, wp1, wp2, ws2, nyq_frec]) / nyq_frec
gains = np.array([-atenuacion, -atenuacion, -ripple, -ripple, -atenuacion, -atenuacion])
gains = 10 ** (gains / 20)

plt.title("Plantilla del filtro")
plt.xlabel("Frecuencia [Hz]")
plt.ylabel("Módulo [dB]")
plt.grid()
plt.axis([0, 100, -60, 5])

plot_plantilla(
    filter_type="bandpass",
    fpass=frecs[[2, 3]] * nyq_frec,
    ripple=ripple,
    fstop=frecs[[1, 4]] * nyq_frec,
    attenuation=atenuacion,
    fs=fs,
)

# Se diseña el filtro FIR

Se utiliza `firwin2` con la window "cosine".

In [ ]:
cant_coeficientes = 10001

# El "filtro" es el numerador de la transferencia. Por ser un FIR, el denominador es 1.
num_win = sig.firwin2(cant_coeficientes, frecs, gains, window="cosine")
den = 1.0

# muestreo el filtro donde me interesa verlo según la plantilla.
w = np.append(np.logspace(-1, 0.8, 250), np.logspace(0.9, 1.6, 250))
w = np.append(w, np.linspace(110, nyq_frec, 100, endpoint=True)) / nyq_frec * np.pi

_, hh_win = sig.freqz(num_win, den, w)

# renormalizo el eje de frecuencia
w = w / np.pi * nyq_frec

plt.plot(w, 20 * np.log10(abs(hh_win)), label="FIR - Coeficientes {:d}".format(num_win.shape[0]))

plt.title("Filtro FIR")
plt.xlabel("Frecuencia [Hz]")
plt.ylabel("Módulo [dB]")
plt.grid()
plt.axis([0, 100, -60, 5])
plt.legend()

plot_plantilla(
    filter_type="bandpass",
    fpass=frecs[[2, 3]] * nyq_frec,
    ripple=ripple,
    fstop=frecs[[1, 4]] * nyq_frec,
    attenuation=atenuacion,
    fs=fs,
)

# Se diseña el filtro IIR

Utilizando `iirdesign` con el parametro "sos" (second order sections).

In [ ]:
# second-order sections (recommended): 'sos'
sos = sig.iirdesign(
    ws=[ws1, ws2],
    wp=[wp1, wp2],
    gpass=ripple,
    gstop=atenuacion,
    fs=fs,
    output="sos",
)

w, h = sig.sosfreqz(sos, worN=100000)
w = w / np.pi * nyq_frec

# Se usa un epsilon para evitar logaritmos de 0
e = 1e-15
plt.plot(
    w,
    20 * np.log10(abs(h)+e),
    label="IIR",
)

plt.title("Filtro IIR")
plt.xlabel("Frecuencia [Hz]")
plt.ylabel("Módulo [dB]")
plt.grid()
plt.axis([0, 100, -60, 5])
plt.legend()

plot_plantilla(
    filter_type="bandpass",
    fpass=frecs[[2, 3]] * nyq_frec,
    ripple=ripple,
    fstop=frecs[[1, 4]] * nyq_frec,
    attenuation=atenuacion,
    fs=fs,
)